## "Basics" K-Means
Um die Daten in Cluster aufzuteilen, kann beispielsweise der [K-Means-Algorithmus](https://de.wikipedia.org/wiki/K-Means-Algorithmus) verwendet werden. Eine Implementation davon befindet sich in der Open-Source-Bibliothek [sklearn](https://scikit-learn.org), s.a. [hier](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Angenommen, wir haben einen Datensatz mit $x_1,\dots,x_n \in \mathbb{R}^r$ Beispielen gegeben. K-Means findet dann $k$ "gute" Zentroiden $c_1,\dots,c_k \in \mathbb{R}^r$ ($k$ wird vorher spezifiziert) und determiniert die Clusterzugehörigkeit über die Distanz eines Datenpunkts zu den Zentroiden. Für einen Datenpunkt $x$ wird das passende Cluster also über $$ \text{cluster}(x) = \mathrm{argmin}_{\ell \in \{1,\dots,k\}} || c_\ell - x ||^2 $$ berechnet. Wenn ein Autoencoder mit der Kodierungsfunktion $f : \mathbb{R}^r \rightarrow \mathbb{R}^s$ und $s << r$ gegeben ist, kann als Featurevektor statt der Waveforms die Kodierung in den K-Means-Algorithmus gegeben werden. Die Idee ist hier, dass durch die Kodierung einerseits unrepräsentative Features entfernt werden, andererseits aber auch die Dimension die Eingabedaten deutlich verkleinert wird, was dem K-Means-Algorithmus zugute kommt.

## Importieren der Bibliotheken und Datensatz laden

In [ ]:
import deepwaveform as dwf
import torch
from sklearn.cluster import KMeans

df = dwf.load_dataset("../data/mit_uferuebergang.txt", wv_cols=list(range(64)))

## Annotieren des Datensatzes mit den Kodierungen eines Autoencoders

In [ ]:
model = dwf.AutoEncoder(dimensions=12)
model.load_state_dict(torch.load("trained_models/autoencoder.pt"))
model.eval()
model.annotate_dataframe(df)
df

## Clustering mit Hilfe von K-Means auf den Waveforms und Autoencoder-Kodierungen

In [ ]:
mat_waveforms = dwf.waveform2matrix(df, wv_cols=list(range(64)))
mat_hidden = dwf.waveform2matrix(df, wv_cols=["hidden_"+str(idx) for idx in range(12)])

kmeans = KMeans(n_clusters=5).fit(mat_waveforms)
centroids_waveforms = kmeans.cluster_centers_
df["cluster_waveforms"] = kmeans.labels_

kmeans = KMeans(n_clusters=5).fit(mat_hidden)
centroids_hidden = kmeans.cluster_centers_
df["cluster_hidden"] = kmeans.labels_

## Vergleich der gefundenen Cluster

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
fig = plt.figure(figsize=(35,9))
ax1 = fig.add_subplot(111, projection="3d", title="Clustering über komplette Waveforms")
ax2 = fig.add_subplot(122, projection="3d", title="Clustering über kodierte Waveforms")
dwf.plot_pcl(df, ax1, targetcol="cluster_waveforms", colormap=cm.Accent)
dwf.plot_pcl(df, ax2, targetcol="cluster_hidden", colormap=cm.Accent)

## Zentroiden der Waveforms und Autoencoder-Kodierungen

In [ ]:
import numpy as np
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,6))

xs = np.arange(centroids_waveforms.shape[1])
for centroid_idx in range(centroids_waveforms.shape[0]):
    ax1.plot(xs, centroids_waveforms[centroid_idx, :], label="cluster %d" % (centroid_idx+1))
ax1.set_title("Zentroiden der Waveforms")
ax1.set_xlabel("Dimension der Zentroiden")
ax1.set_ylabel("Amplitude")
ax1.legend()

clustercount=centroids_hidden.shape[0]
xs = np.arange(centroids_hidden.shape[1])
for centroid_idx in range(clustercount):
    ax2.plot(xs, centroids_hidden[centroid_idx, :], "--x", label="cluster %d" % (centroid_idx+1))
ax2.set_title("Zentroiden der Kodierungen")
ax2.set_xlabel("Dimension der Zentroiden")
ax2.set_ylabel("Amplitude")
ax2.legend()